# XML example and exercise
Study examples of accessing nodes in XML tree structure

Work on exercise to be cleaned and analyzed

Reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html

Data source: http://www.dbis.informatik.uni-goettingen.de/Mondial


In [84]:
import pandas as pd
from pandas import DataFrame,Series
from xml.etree import ElementTree as ET


# XML example
--For details about tree traversal and iterators

--See https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [85]:
document_tree=ET.parse(r'C:\Users\abhij\Desktop\data_wrangling_xml\data\mondial_database_less.xml')

In [86]:
# Print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [87]:
#Print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '*'+element.find('name').text+':',
    capital_string=''
    for subelement in element.getiterator('city'):
        capital_string+=subelement.find('name').text+','
    print capital_string[:-2]

*Albania: Tirana,Shkodër,Durrës,Vlorë,Elbasan,Korç
*Greece: Komotini,Kavala,Athina,Peiraias,Peristeri,Acharnes,Patra,Kozani,Kerkyra,Ioannina,Thessaloniki,Iraklio,Chania,Ermoupoli,Rhodes,Tripoli,Lamia,Chalkida,Larissa,Volos,Mytilini,Karye
*Macedonia: Skopje,Kumanov
*Serbia: Beograd,Novi Sad,Ni
*Montenegro: Podgoric
*Kosovo: Prishtin
*Andorra: Andorra la Vell


# XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html

Find:

# Task 1:-10 countries with the lowest infant mortality rates


In [88]:
doc=ET.parse(r'C:\Users\abhij\Desktop\data_wrangling_xml\data\mondial_database.xml')

In [89]:
#initiate an empty dictionary to store Key:Country and Value:Infant Mortality
infant_dict={}
# for each element in doc for country
for element in doc.iterfind('country'):
    #for each x in element where we got country find its infant_mortality
    for x in element.iterfind('infant_mortality'):
        #now finally we have a empty_dictionary infant_dict waiting to be populated
        #now for each key:country find its corresponding value:infant_mortality rate
        infant_dict[element.find('name').text]=[float(element.find('infant_mortality').text)]

In [90]:
#Convert the infant_dict into a valid pandas dataframe
infant_df=DataFrame.from_dict(infant_dict,orient='index')

In [91]:
#Naming the column 'Infant Mortality Rate'
infant_df.columns=['Infant Mortality Rate']

In [92]:
#Sorting the DataFrame by lowest Infant Mortality Rate for top 10 countries
infant_df.sort_values('Infant Mortality Rate',ascending=True).head(10)

,Infant Mortality Rate
Monaco,1.81
Japan,2.13
Norway,2.48
Bermuda,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


# Task 2:- 10 cities with the largest population

In [93]:
#initiate an empty dictionary for Key:City and Value:Population
city_dict={}
#for each element in doc find country:
for element in doc.iterfind('country'):
    #for each x in element find its city:
    for x in element.iterfind('city'):
        #for each i in x find its population:
        for i in x.iterfind('population'):
            #populate the empty dictionary with values
            city_dict[x.find('name').text]=[int(x.find('population').text)]

In [94]:
#convert to pandas dataframe
city_df=DataFrame.from_dict(city_dict,orient='index')
#name the column
city_df.columns=['Population']
#sort the dataframe
city_df.sort_values('Population',ascending=False).head(10)

,Population
Seoul,10229262
Hong Kong,7055071
Al Qahirah,6053000
Bangkok,5876000
Ho Chi Minh,3924435
Busan,3813814
New Taipei,3722082
Hanoi,3056146
Al Iskandariyah,2917000
Taipei,2626138


# Task 3 :- 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [95]:
#Overview of the approach used
# Step 1 First extract a dictionary containing the Key:Country and Value:Population
# Step 2 Second extract a list containing the followinf elements Country,Ethnic Group,Percentage
# Step 3 Convert the above dicts and list into pandas dataframe
# Step 4 Merge the two dataframes on Country

In [96]:
#Step 1
country_dict={}
for element in doc.iterfind('country'):
    country_dict[element.find('name').text]=[int(element.find('population[last()]').text)]

In [97]:
#convert country_dict to a valid pandas dataframe
country_df=DataFrame.from_dict(country_dict,orient='index')

In [98]:
# dropping the previous columns names so that we can rename it to Country and Population
country_df.reset_index(drop=False,inplace=True)

In [99]:
#naming the columns
country_df.columns=['Country','Population']

In [100]:
country_df.head(4)

,Country,Population
0,Canada,35158304
1,Brazil,202768562
2,Sao Tome and Principe,187356
3,Turkmenistan,5041995


In [101]:
# Step 2 Second extract a list containing the followinf elements Country,Ethnic Group,Percentage

country_list=[]
for element in doc.iterfind('country'):
    c_name=element.find('name').text
    for x in element.iterfind('ethnicgroup'):
        eth_per=x.attrib['percentage']
        eth_name=x.text
        country_list.append([c_name,eth_name,eth_per])

In [102]:
# convert the list into a pandas dataframe
country_list_df=DataFrame(country_list,columns=['Country','Ethnic Group','Percentage'])

In [103]:
country_list_df.head(4)

,Country,Ethnic Group,Percentage
0,Albania,Albanian,95
1,Albania,Greek,3
2,Greece,Greek,93
3,Macedonia,Macedonian,64.2


In [104]:
# Step 4 Merge the two dataframes on Country

country_list_df=country_list_df.merge(country_df[['Country','Population']],on='Country')

In [105]:
country_list_df.head(4)

,Country,Ethnic Group,Percentage,Population
0,Albania,Albanian,95,2800138
1,Albania,Greek,3,2800138
2,Greece,Greek,93,10816286
3,Macedonia,Macedonian,64.2,2059794


In [106]:
# Adding a new column 'Population final '

country_list_df['Population Final']=(country_list_df['Percentage'].astype(float))*country_list_df['Population']

In [107]:
#use groupby function on Ethnic Group and sum it up
c1=country_list_df.groupby('Ethnic Group').sum()

In [108]:
# Final sort
c1.sort_values('Population Final',ascending=False).head(10)

,Population,Population Final
Ethnic Group,,
Han Chinese,1360720000,1.245059e+11
Indo-Aryan,1210854977,8.718156e+10
European,1157295639,4.948722e+10
African,975352746,3.183251e+10
Dravidian,1210854977,3.027137e+10
Mestizo,279743964,1.577344e+10
Bengali,149772364,1.467769e+10
Russian,322438406,1.318570e+10
Japanese,127298000,1.265342e+10


# Task 4:name and country of a) longest river, b) largest lake and c) airport at highest elevation

# a]Longest River

In [109]:
# initiate an empty dict:country_dict to store Key:country and its value:car_code

In [110]:
country_dict={}

for element in doc.iterfind('country'):
    country_dict[element.attrib['car_code']]=element.find('name').text

In [111]:
country_dict

{'A': 'Austria',
 'AFG': 'Afghanistan',
 'AG': 'Antigua and Barbuda',
 'AL': 'Albania',
 'AMSA': 'American Samoa',
 'AND': 'Andorra',
 'ANG': 'Angola',
 'ARM': 'Armenia',
 'ARU': 'Aruba',
 'AUS': 'Australia',
 'AXA': 'Anguilla',
 'AZ': 'Azerbaijan',
 'B': 'Belgium',
 'BD': 'Bangladesh',
 'BDS': 'Barbados',
 'BEN': 'Benin',
 'BERM': 'Bermuda',
 'BF': 'Burkina Faso',
 'BG': 'Bulgaria',
 'BHT': 'Bhutan',
 'BI': 'Burundi',
 'BIH': 'Bosnia and Herzegovina',
 'BOL': 'Bolivia',
 'BR': 'Brazil',
 'BRN': 'Bahrain',
 'BRU': 'Brunei',
 'BS': 'Bahamas',
 'BVIR': 'British Virgin Islands',
 'BY': 'Belarus',
 'BZ': 'Belize',
 'C': 'Cuba',
 'CAM': 'Cameroon',
 'CAYM': 'Cayman Islands',
 'CDN': 'Canada',
 'CEU': 'Ceuta',
 'CH': 'Switzerland',
 'CI': 'Cote dIvoire',
 'CL': 'Sri Lanka',
 'CN': 'China',
 'CO': 'Colombia',
 'COCO': 'Cocos Islands',
 'COM': 'Comoros',
 'COOK': 'Cook Islands',
 'CR': 'Costa Rica',
 'CUR': 'Curacao',
 'CV': 'Cape Verde',
 'CY': 'Cyprus',
 'CZ': 'Czech Republic',
 'D': 'German

In [121]:
r_list=[]
for element in doc.iterfind('river'):
    r_name=element.find('name').text
    r_length=element.find('length')
    if r_length!=None:
        r_len=r_length.text
        for x in element.attrib['country'].split():
            new_code=country_dict[x]
    r_list.append([new_code,x,r_name,float(r_len)])

In [122]:
river_label=['Country','Country Code','River','Length']

In [123]:
river_df=DataFrame(r_list,columns=river_label)

In [124]:
river_df.head(2)

,Country,Country Code,River,Length
0,Iceland,IS,Thjorsa,230.0
1,Iceland,IS,Joekulsa a Fjoellum,206.0


In [126]:
river_df.sort_values('Length',ascending=False).head(1)

,Country,Country Code,River,Length
174,Peru,PE,Amazonas,6448.0


# b] Largest Lake

In [145]:
lake_list=[]
for element in doc.iterfind('lake'):
    lake_name=element.find('name').text
    lake_area=element.find('area')
    if lake_area!=None:
        lake_new_area=lake_area.text
        for x in element.attrib['country'].split():
            new_code=country_dict[x]
    lake_list.append([new_code,x,lake_name,lake_new_area])

In [146]:
lake_df=DataFrame(lake_list,columns=['Country','Country Code','Lake Name','Lake Area'])

In [154]:
lake_df1=lake_df.apply(pd.to_numeric,errors='ignore')

lake_df1.head(3)

,Country,Country Code,Lake Name,Lake Area
0,Finland,SF,Inari,1040.0
1,Finland,SF,Oulujaervi,928.0
2,Finland,SF,Kallavesi,472.0


In [156]:
lake_df1.sort_values('Lake Area',ascending=False).head(1)

,Country,Country Code,Lake Name,Lake Area
54,Turkmenistan,TM,Caspian Sea,386400.0


# c] Airport at Highest Elevation

In [171]:
airport_empty_list=[]

for element in doc.iterfind('airport'):
    a_name=element.find('name').text
    a_elevation=element.find('elevation')
    if a_elevation!=None:
        a_elev=a_elevation.text
        for x in element.attrib['country'].split():
            new_code=country_dict[x]
    airport_empty_list.append([new_code,x,a_name,a_elev])

In [172]:
air_df=DataFrame(airport_empty_list,columns=['Country','Country Code','Airport','Elevation'])

In [174]:
air_df1=air_df.apply(pd.to_numeric,errors='ignore')
air_df1.sort_values('Elevation',ascending=False).head(1)

,Country,Country Code,Airport,Elevation
80,Bolivia,BOL,El Alto Intl,4063.0
